In [14]:
from IPython.display import display, Javascript, HTML, clear_output, IFrame
from ipywidgets import interact, interactive, fixed, interact_manual, AppLayout, GridspecLayout
import ipywidgets as widgets
from src.gradcam import GradCAM, overlay_gradCAM
from src.guidedBackprop import GuidedBackprop, deprocess_image
from src.utils import preprocess, predict, SAMPLE_DIR, array2bytes, DECODE
from src.models import load_ResNet50PlusFC, load_VanilaResNet50
import os, cv2
import matplotlib.pyplot as plt
INV_MAP = {"Cat": 0, "Dog": 1}

In [15]:
# Header
header = widgets.HTML('<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guided GradCAM</h1></center></font>',
                      layout=widgets.Layout(height='auto'))
# Logo
logo = widgets.Image(
    value=open("./assets/illustrations/VietAIlogo.png", "rb").read(),
    format='png',
    width='auto',
    height='auto',
    align="center-align"
)

# Dropdowns
def on_change_im(change):
    if change['type'] == "change" and change["name"] == "value":
        img = cv2.imread(os.path.join(SAMPLE_DIR,change["new"]))
        im_arr = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        logo = widgets.Image(
            value=array2bytes(im_arr),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
        grid[3:13, 9:21] = logo
def on_change_model(change):
    if change['type'] == "change" and change["name"] == "value":
        chosen_model =  widgets.HTML("<p>Model %s loaded." % change["new"])
        grid[2,:8] = chosen_model
        
im_ls = ["--Select"] + os.listdir("./assets/samples")
im_ls.sort()
model_ls = ["--Select","VanilaResNet50", "ResNet50PlusFC"]
class_ls = ["--Select","Cat", "Dog"]
models = widgets.Dropdown(options=model_ls, description="Model",layout={'width':'auto'}, disabled=False)
imgs = widgets.Dropdown(options=im_ls, description="Image", layout={'width':'auto'}, disabled=False)
classes = widgets.Dropdown(options=class_ls, description="Class", layout={'width':'auto'}, disabled=False)
imgs.observe(on_change_im)
models.observe(on_change_model)
# Notes
# note = widgets.HTML("<p><b>Modes:</b><br>- VanilaResNet50 is retrained ResNet50 model on Dog vs. Cat dataset<br>"
#                     "- ResNet50PlusFC is the model with 2 FC layers added to ResNet50, retrained on Dog vs. Cat dataset"
#                     "<br><b>Classes: </b>If not specified, GradCAM & Guided GradCAM will be calculated based on the predicted class</p>")

# button
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))
pred_but = create_expanded_button("Show GradCAM & Guided GradCAM","info")

# Layouts
grid = GridspecLayout(20, 30, height='700px')
grid[0,:] = header
grid[1,:8] = models
grid[1,9:18] = imgs
grid[1,19:27] = classes
grid[3:13, 9:21] = logo
grid[19, 10:20] = pred_but
# grid[17:,:] = note
display(grid)

def showCAMs(img, x, GradCAM, GuidedBP, chosen_class, upsample_size):
    cam3 = GradCAM.compute_heatmap(image=x, classIdx=chosen_class, upsample_size=upsample_size)
    new_img = overlay_gradCAM(img, cam3)
    new_img = cv2.cvtColor(new_img, cv2.COLOR_BGR2RGB)
    gb = GuidedBP.guided_backprop(x, upsample_size)
    guided_gradcam = deprocess_image(gb * cam3)
    guided_gradcam = cv2.cvtColor(guided_gradcam, cv2.COLOR_BGR2RGB)
    gc = widgets.Image(
            value=array2bytes(new_img),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    ggc = widgets.Image(
            value=array2bytes(guided_gradcam),
            format='png',
            width='auto',
            height='auto',
            align="center-align"
        )
    
    grid[3:13, 9:21] = widgets.HTML("")
    grid[3:13, :14] = gc
    grid[3:13, 16:] = ggc
    
def check_button(sender):
    if models.value == "VanilaResNet50":
        model = load_VanilaResNet50()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model,  layerName="conv5_block3_out")
    elif models.value == "ResNet50PlusFC":
        model = load_ResNet50PlusFC()
        gradCAM = GradCAM(model=model, layerName="conv5_block3_out")
        guidedBP = GuidedBackprop(model=model,  layerName="conv5_block3_out")
    img = cv2.imread(os.path.join(SAMPLE_DIR,imgs.value))
    upsample_size = (img.shape[1],img.shape[0])
    x = preprocess(imgs.value)
    pred, prob = predict(model,x)
    if classes.value == "--Select":
        classIdx = pred
    else:
        classIdx = INV_MAP[classes.value]
    
    grid[2,9:18] = widgets.HTML("<span>Predicted: {}<span>".format(DECODE[pred]))
        
    showCAMs(img, x, gradCAM, guidedBP, classIdx, upsample_size)

pred_but.on_click(check_button)

GridspecLayout(children=(HTML(value='<font color="#1f77b4" face="sans-serif"><center><h1>DEMO GradCAM and Guid…